In [7]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import pandas as pd

# Load the preprocessed evaluation dataset
eval_df = pd.read_csv("/content/drive/MyDrive/phishing_data/url_model/PhiUSIIL_Phishing_URL_Eval_Preprocessed.csv")

# Extract texts and labels
eval_texts = eval_df["url"].tolist()
eval_labels = eval_df["label"].tolist()


In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def batch_tokenize(texts, labels, tokenizer, batch_size=10000):
    encodings = {"input_ids": [], "attention_mask": []}
    all_labels = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        batch_labels = labels[i:i+batch_size]
        batch_enc = tokenizer(batch_texts, truncation=True, padding=True, return_tensors="pt")
        encodings["input_ids"].extend(batch_enc["input_ids"])
        encodings["attention_mask"].extend(batch_enc["attention_mask"])
        all_labels.extend(batch_labels)
    return encodings, all_labels

eval_encodings, eval_labels_final = batch_tokenize(eval_texts, eval_labels, tokenizer)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [11]:
import torch

class PhishingDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        return {
            key: self.encodings[key][idx] for key in self.encodings
        } | {
            "labels": torch.tensor(self.labels[idx])
        }
    def __len__(self):
        return len(self.labels)

eval_dataset = PhishingDataset(eval_encodings, eval_labels_final)

In [12]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [13]:
from transformers import AutoModelForSequenceClassification, Trainer

model_paths = {
    "Phase 1": "/content/drive/MyDrive/phishing_model_v1_after_phase1",
    "Phase 3": "/content/drive/MyDrive/phishing_model_v3_after_phase3"
}

for name, path in model_paths.items():
    print(f"\n🔍 Evaluating {name} model...")
    model = AutoModelForSequenceClassification.from_pretrained(path)
    trainer = Trainer(
        model=model,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics
    )
    results = trainer.evaluate()
    print(f"\n Evaluation Results for {name}")
    for metric, value in results.items():
        print(f"{metric}: {value:.4f}")



🔍 Evaluating Phase 1 model...


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: gayathriharshila (gayathriharshila-university-of-moratuwa) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin



 Evaluation Results for Phase 1
eval_loss: 2.8182
eval_model_preparation_time: 0.0014
eval_accuracy: 0.5831
eval_precision: 0.5817
eval_recall: 0.9643
eval_f1: 0.7257
eval_runtime: 2878.9272
eval_samples_per_second: 81.9040
eval_steps_per_second: 10.2380

🔍 Evaluating Phase 3 model...



 Evaluation Results for Phase 3
eval_loss: 2.8182
eval_model_preparation_time: 0.0014
eval_accuracy: 0.5831
eval_precision: 0.5817
eval_recall: 0.9643
eval_f1: 0.7257
eval_runtime: 2880.8194
eval_samples_per_second: 81.8500
eval_steps_per_second: 10.2310
